In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


### Load Dataset

In [2]:
df = pd.read_csv("C:/Users/HP/Desktop/DERSLER/Derin Öğrenme Lab/lab3_12032025/BankNote_Authentication.csv")


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   variance  1372 non-null   float64
 1   skewness  1372 non-null   float64
 2   curtosis  1372 non-null   float64
 3   entropy   1372 non-null   float64
 4   class     1372 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 53.7 KB


In [4]:
df.head(6)

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0
5,4.36840,9.6718,-3.9606,-3.16250,0


In [5]:
df.isnull().sum()

variance    0
skewness    0
curtosis    0
entropy     0
class       0
dtype: int64

In [6]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
X, y = df.iloc[:, :-1], df.iloc[:, -1]

X = X.to_numpy()
y = y.to_numpy().reshape(-1, 1)

In [7]:
# Veriyi eğitim ve test kümelerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Veri normalizasyonu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:

class MLP:
    def __init__(self, input_size, hidden_layers, output_size=1, learning_rate=0.1, activation="relu"):
        self.hidden_layers = hidden_layers
        self.learning_rate = learning_rate
        self.activation_function = activation
        
        self.weights = []
        self.biases = []
        
        layer_sizes = [input_size] + hidden_layers + [output_size]
        for i in range(len(layer_sizes) - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.1)
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

    def activation(self, x):
        if self.activation_function == "tanh":
            return np.tanh(x)
        elif self.activation_function == "relu":
            return np.maximum(0, x)

    def activation_derivative(self, x):
        if self.activation_function == "tanh":
            return 1 - np.tanh(x) ** 2
        elif self.activation_function == "relu":
            return np.where(x > 0, 1, 0)

    def forward_propagation(self, X):
        activations = [X]
        inputs = []

        for i in range(len(self.hidden_layers)):
            z = np.dot(activations[-1], self.weights[i]) + self.biases[i]
            inputs.append(z)
            activations.append(self.activation(z))

        z_output = np.dot(activations[-1], self.weights[-1]) + self.biases[-1]
        inputs.append(z_output)
        activations.append(1 / (1 + np.exp(-z_output)))

        return activations, inputs

    def compute_cost(self, y_true, y_pred):
        m = y_true.shape[0]
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

    def back_propagation(self, activations, inputs, y_true):
        m = y_true.shape[0]
        grads_w = [np.zeros_like(w) for w in self.weights]
        grads_b = [np.zeros_like(b) for b in self.biases]

        dz = activations[-1] - y_true.reshape(-1, 1)
        grads_w[-1] = np.dot(activations[-2].T, dz) / m
        grads_b[-1] = np.sum(dz, axis=0, keepdims=True) / m

        for i in range(len(self.hidden_layers) - 1, -1, -1):
            dz = np.dot(dz, self.weights[i + 1].T) * self.activation_derivative(inputs[i])
            grads_w[i] = np.dot(activations[i].T, dz) / m
            grads_b[i] = np.sum(dz, axis=0, keepdims=True) / m

        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * grads_w[i]
            self.biases[i] -= self.learning_rate * grads_b[i]

    def train(self, X_train, y_train, epochs=200):
        for epoch in range(epochs):
            activations, inputs = self.forward_propagation(X_train)
            cost = self.compute_cost(y_train, activations[-1])
            self.back_propagation(activations, inputs, y_train)
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}: Cost = {cost:.4f}")


    def predict(self, X):
        activations, _ = self.forward_propagation(X)
        return (activations[-1] >= 0.5).astype(int)

models = {
    "tanh_1": MLP(input_size=X.shape[1], hidden_layers=[5], activation="tanh"),
    "tanh_2": MLP(input_size=X.shape[1], hidden_layers=[5, 5], activation="tanh"),
    "relu_1": MLP(input_size=X.shape[1], hidden_layers=[5], activation="relu"),
    "relu_2": MLP(input_size=X.shape[1], hidden_layers=[5, 5], activation="relu")
}



In [9]:
from sklearn.metrics import accuracy_score, classification_report


### Compute Cost

In [10]:
results = []

for activation in ["tanh", "relu"]:
    for layers in [[5], [5, 5]]:
        print(f"Training MLP with {activation.upper()} activation and {len(layers)} hidden layer(s)...")
        mlp = MLP(input_size=X.shape[1], hidden_layers=layers, learning_rate=0.1, activation=activation)
        mlp.train(X_train, y_train, epochs=400)
        y_pred = mlp.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        results.append((activation, len(layers), accuracy, report))
    

Training MLP with TANH activation and 1 hidden layer(s)...
Epoch 10: Cost = 0.6615
Epoch 20: Cost = 0.6343
Epoch 30: Cost = 0.5991
Epoch 40: Cost = 0.5573
Epoch 50: Cost = 0.5118
Epoch 60: Cost = 0.4657
Epoch 70: Cost = 0.4207
Epoch 80: Cost = 0.3777
Epoch 90: Cost = 0.3370
Epoch 100: Cost = 0.2994
Epoch 110: Cost = 0.2652
Epoch 120: Cost = 0.2350
Epoch 130: Cost = 0.2090
Epoch 140: Cost = 0.1869
Epoch 150: Cost = 0.1683
Epoch 160: Cost = 0.1528
Epoch 170: Cost = 0.1399
Epoch 180: Cost = 0.1291
Epoch 190: Cost = 0.1199
Epoch 200: Cost = 0.1121
Epoch 210: Cost = 0.1054
Epoch 220: Cost = 0.0996
Epoch 230: Cost = 0.0946
Epoch 240: Cost = 0.0902
Epoch 250: Cost = 0.0863
Epoch 260: Cost = 0.0829
Epoch 270: Cost = 0.0798
Epoch 280: Cost = 0.0770
Epoch 290: Cost = 0.0745
Epoch 300: Cost = 0.0722
Epoch 310: Cost = 0.0702
Epoch 320: Cost = 0.0683
Epoch 330: Cost = 0.0665
Epoch 340: Cost = 0.0649
Epoch 350: Cost = 0.0635
Epoch 360: Cost = 0.0621
Epoch 370: Cost = 0.0608
Epoch 380: Cost = 0.0596


### Comparison of Models

In [11]:
print("\nFinal Comparison of Models:")
print("=" * 60)
for activation, num_layers, accuracy, report in results:
    print(f"Activation: {activation.upper()}, Hidden Layers: {num_layers}")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(report)
    print("-" * 60)



Final Comparison of Models:
Activation: TANH, Hidden Layers: 1
Accuracy: 0.9891
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       153
           1       0.98      1.00      0.99       122

    accuracy                           0.99       275
   macro avg       0.99      0.99      0.99       275
weighted avg       0.99      0.99      0.99       275

------------------------------------------------------------
Activation: TANH, Hidden Layers: 2
Accuracy: 0.9927
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       153
           1       0.98      1.00      0.99       122

    accuracy                           0.99       275
   macro avg       0.99      0.99      0.99       275
weighted avg       0.99      0.99      0.99       275

------------------------------------------------------------
Activation: RELU, Hidden Layers: 1
Accuracy: 